In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import itertools
import datetime

In [2]:
path = "/content/drive/MyDrive/data/df2_h.csv"
df = pd.read_csv(path, parse_dates=[0, 1, 2])

In [3]:
def delete_empty_activities(data):
  return data.loc[df.begin != df.end]

In [4]:
df_weekend_1 = df[df.date.dt.dayofweek >= 5]
df.head()

,date,begin,end,isprod
0,2022-05-20,2022-05-20 17:13:00,2022-05-20 17:31:00,0
1,2022-05-20,2022-05-20 16:32:00,2022-05-20 17:13:00,0
2,2022-05-20,2022-05-20 16:16:00,2022-05-20 16:32:00,0
3,2022-05-20,2022-05-20 15:35:00,2022-05-20 16:16:00,1
4,2022-05-20,2022-05-20 15:28:00,2022-05-20 15:35:00,0


In [5]:
df.head(10)

,date,begin,end,isprod
0,2022-05-20,2022-05-20 17:13:00,2022-05-20 17:31:00,0
1,2022-05-20,2022-05-20 16:32:00,2022-05-20 17:13:00,0
2,2022-05-20,2022-05-20 16:16:00,2022-05-20 16:32:00,0
3,2022-05-20,2022-05-20 15:35:00,2022-05-20 16:16:00,1
4,2022-05-20,2022-05-20 15:28:00,2022-05-20 15:35:00,0
5,2022-05-20,2022-05-20 15:27:00,2022-05-20 15:28:00,2
6,2022-05-20,2022-05-20 15:17:00,2022-05-20 15:24:00,2
7,2022-05-20,2022-05-20 15:08:00,2022-05-20 15:17:00,1
8,2022-05-20,2022-05-20 13:16:00,2022-05-20 15:08:00,1
9,2022-05-20,2022-05-20 12:55:00,2022-05-20 13:16:00,2


In [6]:
def add_empty_ranges(data):
  first = data.begin.shift(periods=1) - data.end > datetime.timedelta(0)
  second = data.begin - data.end.shift(periods=-1) > datetime.timedelta(0)
  df_1 = data.loc[first]
  k = pd.DataFrame(columns=['date',	'begin',	'end',	'isprod'])
  for i, row in df_1.iterrows():
    tmp = [data.loc[i - 1].date, row.end, data.loc[i - 1].begin, 2]
    k.loc[len(k)] = tmp
  return k

In [7]:
x = add_empty_ranges(df)
df = pd.concat([df, x])
print(df)

          date               begin                 end isprod
0   2022-05-20 2022-05-20 17:13:00 2022-05-20 17:31:00      0
1   2022-05-20 2022-05-20 16:32:00 2022-05-20 17:13:00      0
2   2022-05-20 2022-05-20 16:16:00 2022-05-20 16:32:00      0
3   2022-05-20 2022-05-20 15:35:00 2022-05-20 16:16:00      1
4   2022-05-20 2022-05-20 15:28:00 2022-05-20 15:35:00      0
..         ...                 ...                 ...    ...
968 2021-10-25 2021-10-25 11:32:00 2021-10-25 11:33:00      2
969 2021-10-25 2021-10-25 10:03:00 2021-10-25 10:07:00      2
970 2021-10-25 2021-10-25 08:57:00 2021-10-25 08:59:00      2
971 2021-10-25 2021-10-25 08:40:00 2021-10-25 08:43:00      2
972 2021-10-25 2021-10-25 08:18:00 2021-10-25 08:19:00      2

[5966 rows x 4 columns]


In [8]:
df = delete_empty_activities(df)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5849 entries, 0 to 972
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5849 non-null   datetime64[ns]
 1   begin   5849 non-null   datetime64[ns]
 2   end     5849 non-null   datetime64[ns]
 3   isprod  5849 non-null   object        
dtypes: datetime64[ns](3), object(1)
memory usage: 228.5+ KB


In [10]:
df.begin = df.begin.apply(lambda x: x.round(freq = '5min'))
df.end = df.end.apply(lambda x: x.round(freq = '5min'))

In [11]:
df.head()

,date,begin,end,isprod
0,2022-05-20,2022-05-20 17:15:00,2022-05-20 17:30:00,0
1,2022-05-20,2022-05-20 16:30:00,2022-05-20 17:15:00,0
2,2022-05-20,2022-05-20 16:15:00,2022-05-20 16:30:00,0
3,2022-05-20,2022-05-20 15:35:00,2022-05-20 16:15:00,1
4,2022-05-20,2022-05-20 15:30:00,2022-05-20 15:35:00,0


In [12]:
df['ranges'] = df[['begin', 'end']].agg(lambda x: pd.date_range(start=x[0], end=x[1], freq='5min'),  axis=1)
df = delete_empty_activities(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5292 entries, 0 to 971
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5292 non-null   datetime64[ns]
 1   begin   5292 non-null   datetime64[ns]
 2   end     5292 non-null   datetime64[ns]
 3   isprod  5292 non-null   object        
 4   ranges  5292 non-null   object        
dtypes: datetime64[ns](3), object(2)
memory usage: 248.1+ KB


In [13]:
df.loc[0].ranges

DatetimeIndex(['2022-05-20 17:15:00', '2022-05-20 17:20:00',
               '2022-05-20 17:25:00', '2022-05-20 17:30:00'],
              dtype='datetime64[ns]', freq='5T')

In [14]:
df.ranges

0      DatetimeIndex(['2022-05-20 17:15:00', '2022-05...
1      DatetimeIndex(['2022-05-20 16:30:00', '2022-05...
2      DatetimeIndex(['2022-05-20 16:15:00', '2022-05...
3      DatetimeIndex(['2022-05-20 15:35:00', '2022-05...
4      DatetimeIndex(['2022-05-20 15:30:00', '2022-05...
                             ...                        
964    DatetimeIndex(['2021-10-25 15:15:00', '2021-10...
966    DatetimeIndex(['2021-10-25 13:00:00', '2021-10...
968    DatetimeIndex(['2021-10-25 11:30:00', '2021-10...
970    DatetimeIndex(['2021-10-25 08:55:00', '2021-10...
971    DatetimeIndex(['2021-10-25 08:40:00', '2021-10...
Name: ranges, Length: 5292, dtype: object

In [15]:
df.ranges = df.ranges.map(lambda x: list(zip(x[:-1:], x[1::])))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5292 entries, 0 to 971
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5292 non-null   datetime64[ns]
 1   begin   5292 non-null   datetime64[ns]
 2   end     5292 non-null   datetime64[ns]
 3   isprod  5292 non-null   object        
 4   ranges  5292 non-null   object        
dtypes: datetime64[ns](3), object(2)
memory usage: 377.1+ KB


In [16]:
df = df.explode('ranges', ignore_index=True)


In [17]:
df[df.isnull().any(1)]

,date,begin,end,isprod,ranges
30511,2021-12-01,2021-12-01 21:45:00,2021-12-01 21:00:00,1,NaN


In [18]:
df = df.drop(columns=['begin','end'])
df = df.dropna()

In [19]:
df.head(-10)

,date,isprod,ranges
0,2022-05-20,0,"(2022-05-20 17:15:00, 2022-05-20 17:20:00)"
1,2022-05-20,0,"(2022-05-20 17:20:00, 2022-05-20 17:25:00)"
2,2022-05-20,0,"(2022-05-20 17:25:00, 2022-05-20 17:30:00)"
3,2022-05-20,0,"(2022-05-20 16:30:00, 2022-05-20 16:35:00)"
4,2022-05-20,0,"(2022-05-20 16:35:00, 2022-05-20 16:40:00)"
...,...,...,...
60801,2021-10-26,2,"(2021-10-26 08:35:00, 2021-10-26 08:40:00)"
60802,2021-10-26,2,"(2021-10-26 08:40:00, 2021-10-26 08:45:00)"
60803,2021-10-26,2,"(2021-10-26 08:45:00, 2021-10-26 08:50:00)"
60804,2021-10-26,2,"(2021-10-26 08:50:00, 2021-10-26 08:55:00)"


In [20]:
df_weekend = df[df.date.dt.dayofweek >= 5]
df_weekend

,date,isprod,ranges
831,2022-05-15,0,"(2022-05-15 21:30:00, 2022-05-15 21:35:00)"
832,2022-05-15,0,"(2022-05-15 21:35:00, 2022-05-15 21:40:00)"
833,2022-05-15,0,"(2022-05-15 21:40:00, 2022-05-15 21:45:00)"
834,2022-05-15,0,"(2022-05-15 21:45:00, 2022-05-15 21:50:00)"
835,2022-05-15,0,"(2022-05-15 21:50:00, 2022-05-15 21:55:00)"
...,...,...,...
60280,2021-10-30,2,"(2021-10-30 09:35:00, 2021-10-30 09:40:00)"
60281,2021-10-30,2,"(2021-10-30 09:40:00, 2021-10-30 09:45:00)"
60282,2021-10-30,2,"(2021-10-30 09:45:00, 2021-10-30 09:50:00)"
60283,2021-10-30,2,"(2021-10-30 09:50:00, 2021-10-30 09:55:00)"


In [21]:
df_weekday = df[df.date.dt.dayofweek < 5]
df_weekday

,date,isprod,ranges
0,2022-05-20,0,"(2022-05-20 17:15:00, 2022-05-20 17:20:00)"
1,2022-05-20,0,"(2022-05-20 17:20:00, 2022-05-20 17:25:00)"
2,2022-05-20,0,"(2022-05-20 17:25:00, 2022-05-20 17:30:00)"
3,2022-05-20,0,"(2022-05-20 16:30:00, 2022-05-20 16:35:00)"
4,2022-05-20,0,"(2022-05-20 16:35:00, 2022-05-20 16:40:00)"
...,...,...,...
60811,2021-10-25,2,"(2021-10-25 15:15:00, 2021-10-25 15:20:00)"
60812,2021-10-25,2,"(2021-10-25 13:00:00, 2021-10-25 13:05:00)"
60813,2021-10-25,2,"(2021-10-25 11:30:00, 2021-10-25 11:35:00)"
60814,2021-10-25,2,"(2021-10-25 08:55:00, 2021-10-25 09:00:00)"
